In [1]:
import ipywidgets as widgets
from IPython.display import display
from PIL import Image
import io
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

#create button widget
upload_button = widgets.FileUpload(description='Upload Image')
submit_button = widgets.Button(description='Submit')
output = widgets.Output()

# Define submit button click event handler
def on_submit_button_click(button):
    with output:
        if upload_button.value:
            uploaded_files = list(upload_button.value.values())
            if uploaded_files:
                image_bytes = uploaded_files[0]['content']
                image = Image.open(io.BytesIO(image_bytes))
                print("Image uploaded successfully")
                detect_shapes(image)
            else:
                print('No image uploaded. Please upload an image.')
        else:
            print('No image uploaded. Please upload an image.')

# Assign event handler to the submit button
submit_button.on_click(on_submit_button_click)

# Display UI elements
display(upload_button)
display(submit_button)
display(output)

def detect_shapes(image):
    # Convert Image to NumPy array
    img_np = np.array(image)

    # Process the image
    img = cv2.resize(img_np, (500, 500))

    # Convert to grayscale if the image is not already in grayscale
    if len(img.shape) == 3 and img.shape[2] == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img  # Image is already in grayscale

    _, threshold = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(
        threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        approx = cv2.approxPolyDP(
            contour, 0.02 * cv2.arcLength(contour, True), True)
        cv2.drawContours(img, [contour], 0, (0, 0, 255), 5)
        M = cv2.moments(contour)
        if M['m00'] != 0.0:
            x = int(M['m10'] / M['m00'])
            y = int(M['m01'] / M['m00'])
            if len(approx) == 3:
                cv2.putText(img, 'Triangle', (x, y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
            elif len(approx) == 4:
                cv2.putText(img, 'Quadrilateral', (x, y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
            elif len(approx) == 5:
                cv2.putText(img, 'Pentagon', (x, y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
            elif len(approx) == 6:
                cv2.putText(img, 'Hexagon', (x, y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
            else:
                cv2.putText(img, 'Circle', (x, y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)

    cv2_imshow(img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()



FileUpload(value={}, description='Upload Image')

Button(description='Submit', style=ButtonStyle())

Output()